In [ ]:
import time
from threading import Thread
import os

db_value = 0

def increase(thread_name):
    global db_value
    
    print(f"Thread {thread_name} with init value: {db_value}")
    local_copy = db_value
    
    # processing
    local_copy +=1
    time.sleep(0.1)
    db_value = local_copy
    print(f"Done thread {thread_name}!")
    
        
if __name__ == '__main__':
    print(f'start value: {db_value}')
    
    thread1 = Thread(target=increase, args=(1,))
    thread2 = Thread(target=increase, args=(2,))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    print(f"End value: {db_value}")

    print('end main')



Lý do có 2 thread chạy mà end value lại là 1 là do khi đang chạy thread 1 đến dòng `time.sleep(0.1)` khi đó hệ thống sẽ chạy luôn thread 2 => lúc đó thì `db_value` vẫn là 0 => sau khi chạy 2 thread thì giá trị cuối cùng của `db_value` là 1.

Có thể nhận ra điều này vì nhìn console in ra thì dòng `Thread 2 with init...` xuất hiện trước khi `Done thread 1`

Có một cách để ngăn việc chạy sang thread khác trước khi thread này chạy chưa xong là dòng `Lock` như sau:

In [2]:
import time
from threading import Thread, Lock
import os

db_value = 0

def increase(thread_name, lock):
    global db_value
    
    # lock.acquire()
    # print(f"Thread {thread_name} with init value: {db_value}")
    # local_copy = db_value
    
    # # processing
    # local_copy +=1
    # time.sleep(0.1)
    # db_value = local_copy
    # print(f"Done thread {thread_name}!")
    # lock.release()
    
    # HOặc
    with lock:
        print(f"Thread {thread_name} with init value: {db_value}")
        local_copy = db_value

        # processing
        local_copy +=1
        time.sleep(0.1)
        db_value = local_copy
        print(f"Done thread {thread_name}!")
    
        
if __name__ == '__main__':
    print(f'start value: {db_value}')
    lock = Lock()
    
    thread1 = Thread(target=increase, args=(1,lock,))
    thread2 = Thread(target=increase, args=(2,lock,))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    print(f"End value: {db_value}")

    print('end main')



start value: 0
Thread 1 with init value: 0
Done thread 1!
Thread 2 with init value: 1
Done thread 2!
End value: 2
end main


Một cách khác để thực hiện các task sao cho theo đúng thứ tự là sử dụng `Queue`

In [3]:
from queue import Queue
from threading import Thread, Lock, current_thread
import time

def worker(q):
    while True:
        value = q.get()
        # processing..
        print(f'in {current_thread().name} got {value}')
        q.task_done()

if __name__ == "__main__":
    
    q = Queue()
    
    num_threads = 10
    
    for i in range(num_threads):
        thread = Thread(target=worker, args=(q,))
        thread.daemon = True
        thread.start()
    
    for i in range(1, 10):
        q.put(i)
        
    q.join()
    print('end main')

in Thread-18 got 1
in Thread-18 got 2
in Thread-18 got 3
in Thread-18 got 4
in Thread-18 got 5
in Thread-18 got 6
in Thread-18 got 7
in Thread-18 got 8
in Thread-18 got 9
end main
